In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import tqdm as tqdm
from torch.optim import Adam
from torch.autograd import Variable
from torchvision.transforms import transforms
import pathlib
import nibabel as nib
from torch.utils.data import Dataset, DataLoader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

In [1]:
Train_voor = []

Train_na = []



path = pathlib.Path(__file__).parent

for mouse in ["M03", "M04", "M05", "M06", "M07", "M08"]:

    for timestamp in ["-001h", "024h"]:

        if timestamp == "-001h":

            path_ct = path / f"processed/{mouse}_{timestamp}_CT280.img"

            Train_voor.append(nib.load(path_ct).get_fdata())

        else:

            path_ct = path / f"processed/{mouse}_{timestamp}_CT280.img"

            Train_na.append(nib.load(path_ct).get_fdata())

NameError: name 'pathlib' is not defined

In [ ]:
class conv_block(nn.Module): #dit is 1 blok van 2 convs gevolgd door een relu
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels) #is dit noodzakelijk en waarom doet men dit?

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU() #evt leaky ReLu??

    def forward(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        return x


class encoder_block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv = conv_block(in_channels, out_channels)
        self.pool = nn.MaxPool2d((2, 2))

    def forward(self, inputs):
        x = self.conv(inputs)
        p = self.pool(x)

        return x, p

class decoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.up = nn.ConvTranspose2d(in_c, out_c, kernel_size=2, stride=2, padding=0)
        self.conv = conv_block(out_c+out_c, out_c)

    def forward(self, inputs, skip):
        x = self.up(inputs)
        x = torch.cat([x, skip], axis=1)
        x = self.conv(x)
        return x

In [ ]:
class UNet(nn.Module):
    def __init__(self):
        super().__init__() #residuals nog implementeren.

        """ Encoder """
        self.e1 = encoder_block(1, 64)
        self.e2 = encoder_block(64, 128)
        self.e3 = encoder_block(128, 256)
        self.e4 = encoder_block(256, 512)

        """ Bottleneck """
        self.b = conv_block(512, 1024) #hoe beslis je eig hoeveel features je wilt per layer?

        """ Decoder """
        self.d1 = decoder_block(1024, 512)
        self.d2 = decoder_block(512, 256)
        self.d3 = decoder_block(256, 128)
        self.d4 = decoder_block(128, 64)

        """ Classifier """
        self.outputs = nn.Conv2d(64, 1, kernel_size=1, padding=0)

    def forward(self, inputs):
        """ Encoder """
        s1, p1 = self.e1(inputs)
        s2, p2 = self.e2(p1)
        s3, p3 = self.e3(p2)
        s4, p4 = self.e4(p3)

        """ Bottleneck """
        b = self.b(p4)

        """ Decoder """
        d1 = self.d1(b, s4)
        d2 = self.d2(d1, s3)
        d3 = self.d3(d2, s2)
        d4 = self.d4(d3, s1)

        outputs = self.outputs(d4)
        return outputs

In [ ]:
model = UNet().to(device)
optimizer = Adam(model.parameters(),lr=0.01,weight_decay=0.001)
loss_function = nn.MSELoss()

In [ ]:
num_epochs = 15
# 6 muizen, 2 time instances, 154 slices, (242,121)

In [ ]:
transformer = transforms.Compose([transforms.ToTensor(),transforms.Normalize(0.5,0.5)]) #is horizontal flip nodig ?
class MuizenDataset(Dataset):

    def __init__(self,data_voor,data_na):
        super().__init__()
        self.data = data #vanwege de kleine dataset laden we het gewoon helemaal in memory

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        input = self.data_voor[index]
        target = self.data_na[index]
        input = torch.tensor(input)
        target = torch.tensor(target)
        return input, target

train_loader = DataLoader(MuizenDataset(train_data,transform=transformer),batch_size=50,shuffle=True)
test_loader = DataLoader(MuizenDataset(test_data,transform=transformer),batch_size=50,shuffle=True)

In [ ]:
## TRAINING
for epoch in tqdm(range(num_epochs)):  #we itereren meerdere malen over de data tot convergence?
    model.train()

    for i, (image1,image2) in enumerate(train_loader): #wat is een handige manier om dit in te lezen?
        if torch.cuda.is_available():
            image_voor=Variable(image1.cuda())
            image_na=Variable(image2.cuda())

        optimizer.zero_grad()
        image_na_pred = model(image_voor)
        loss = loss_function(image_na_pred,image_na) #vergelijk predicted na image met de echte na image
        loss.backward()
        optimizer.step()

        if i%100==0:
            print('Step: '+ str(i)+'loss: '+str(loss))

In [ ]:
## TESTING
model.eval()
test_acc = []
for i, (image1,image2) in enumerate(train_loader): #wat is een handige manier om dit in te lezen?
    if torch.cuda.is_available():
        image_voor=Variable(image1.cuda())
        image_na=Variable(image2.cuda())

    image_na_pred = model(image_voor)
    loss = loss_function(image_na_pred,image_na) #vergelijk predicted na image met de echte na image
    test_acc.append(loss)
    if i%100==0:
        print('Step: '+ str(i)+'loss: '+str(loss))
av_test_acc = np.mean(np.array(test_acc))
print(av_test_acc)